<a href="https://colab.research.google.com/github/aettikang/bigdata_analysis_basic/blob/main/%EA%B8%B0%EB%A7%90%EB%A0%88%ED%8F%AC%ED%8A%B8(%EC%A7%80%EB%8F%84%ED%95%99%EC%8A%B5)_%EB%AC%B8%EC%A0%9C%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_breast_cancer
cancer=load_breast_cancer()
#독립변수
X = cancer.data
#종속변수
y = cancer.target

In [2]:
##문제1) 데이터셋을 트레이닝데이터(X_train, y_train)와 테스트데이터(X_test, y_test)로 분리
##조건
# 1) 종속변수(y) 데이터의 비율을 유지함
# 2) 트레이닝데이터와 테스트데이터 분리 비율은 70:30
# 2) random_state = 1

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=1)


In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

In [6]:
##문제2) 커널의 형태가 rbf, 선형, 다항인 SVM모델을 각각 만들기
from sklearn.svm import SVC

rbf_model = SVC(kernel='rbf')
linear_model = SVC(kernel='linear')
poly_model = SVC(kernel='poly')


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold,GridSearchCV

##문제3) 다음 파이프라인에 base_model을 순서대로 추가하여 실행한 결과를 문제지의 표 형태로 정리하시오.
### 다음 코드를 base_model만 변경하여 총 3번 실행하시면 됩니다.
#### (1) rbf 모델(rbf_model)  (2)선형모델(linear_model)  (3) 다항모델(poly_model)
models = [rbf_model, linear_model, poly_model]

for model in models:
  pipe=Pipeline([('scaler',scaler), ('base_model', model)])

  kfold=KFold(n_splits=4, shuffle=True, random_state=1)

  param_grid={'base_model__C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'base_model__gamma':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'base_model__degree':[1,2,3,4,5]}

  grid_model=GridSearchCV(estimator=pipe, param_grid=param_grid, cv=kfold, n_jobs=-1).fit(X_train, y_train)

  print('모델: ', model)
  print('교차검증 점수 : ', grid_model.best_score_)
  print('최적의 하이퍼 파라메터 조합 :', grid_model.best_params_)
  print()

모델:  SVC()
교차검증 점수 :  0.9823737373737373
최적의 하이퍼 파라메터 조합 : {'base_model__C': 10, 'base_model__degree': 1, 'base_model__gamma': 0.1}

모델:  SVC(kernel='linear')
교차검증 점수 :  0.97739898989899
최적의 하이퍼 파라메터 조합 : {'base_model__C': 10, 'base_model__degree': 1, 'base_model__gamma': 0.0001}

모델:  SVC(kernel='poly')
교차검증 점수 :  0.9849242424242425
최적의 하이퍼 파라메터 조합 : {'base_model__C': 0.0001, 'base_model__degree': 2, 'base_model__gamma': 100}



In [11]:
##문제4)각 모델의 하이퍼파라미터를 사용하여 SVM모델을 만들고, 학습시켜 테스트데이터에 대한 예측값을 생성하시오.
##      예측값과 테스트데이터셋의 종속변수값(groud truth)를 분류결과레포트(classification_report)에 입력하여 모델을 평가하는 표로 정리하시오.

from sklearn.metrics import classification_report

seed = 1
#파라미터 수정부분(아래 값은 예시임, 문제3에서 입력한 3개 모델의 최적의 파라미터 값을 입력해야 함)
svc_params_rbf = {
    'C': 10,
    'gamma': 0.1,
    'random_state': seed
}

svc_params_poly = {
    'C': 1,
    'degree': 3,
    'gamma': 'scale',
    'random_state': seed
}

svc_params_linear = {
    'C': 0.1,
    'random_state': seed
}

kernels_params = {
    'rbf': svc_params_rbf,
    'poly': svc_params_poly,
    'linear': svc_params_linear
}

###답 추가부분
##(1) A부분을 삭제한 후 각 모델의 커널을 입력. 예를 들어 rbf_model의 경우 rbf 입력하면 됨

for kernel, params in kernels_params.items():
    svm_model = SVC(kernel=kernel, **params)

    ##(2)svm_model을 트레이닝 데이터로 학습
    svm_model.fit(X_train, y_train)

    ##(3)테스트데이터셋으로 예측하여 예측값(svm_prediction) 산출
    svm_prediction = svm_model.predict(X_test)

    ##(4)classification_report 함수로 테스트셋과 예측값 비교
    print('모델: ', kernel)
    print(classification_report(y_test, svm_prediction))
    print()

모델:  rbf
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        64
           1       0.63      1.00      0.77       107

    accuracy                           0.63       171
   macro avg       0.31      0.50      0.38       171
weighted avg       0.39      0.63      0.48       171


모델:  poly
              precision    recall  f1-score   support

           0       0.95      0.83      0.88        64
           1       0.90      0.97      0.94       107

    accuracy                           0.92       171
   macro avg       0.93      0.90      0.91       171
weighted avg       0.92      0.92      0.92       171




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


모델:  linear
              precision    recall  f1-score   support

           0       0.94      0.92      0.93        64
           1       0.95      0.96      0.96       107

    accuracy                           0.95       171
   macro avg       0.95      0.94      0.94       171
weighted avg       0.95      0.95      0.95       171


